# MCP Date Server Exercise

This notebook demonstrates how to create and use an MCP Server that provides a tool to get the current date.

Following the pattern from the accounts server example, we'll:
1. Test the MCP server directly
2. Use it with an Agent
3. (Optional) Create a client wrapper for native OpenAI calls

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

### Step 1: Connect to our Date MCP Server and List Available Tools

In [3]:
from agents.mcp import MCPServerStdioParams

# Connect to our date server as an MCP server
# params = MCPServerStdioParams(command="uv", args=["run", "mcp_date_server.py"])

# If you're not using uv, you can use python directly:
params = MCPServerStdioParams(command="python", args=["mcp_date_server.py"])

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()
    print("Available tools:")
    for tool in mcp_tools:
        print(f"  - {tool.name}: {tool.description}")

Available tools:
  - get_current_date: Get the current date and time.

    Returns:
        The current date and time in a human-readable format
  - get_date_with_format: Get the current date with a specific format.

    Args:
        format: The format string (e.g., '%Y-%m-%d', '%B %d, %Y')


### Step 2: Use the MCP Server with an Agent

In [4]:
# Define the agent's instructions and the request
instructions = "You are a helpful assistant that can tell users the current date and time."
request = "What's today's date? Can you also show me the date in ISO format?"
model = "gpt-4o-mini"  # or "gpt-3.5-turbo" for a cheaper option

In [5]:
# Run the agent with the MCP server
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(
        name="date_assistant", 
        instructions=instructions, 
        model=model, 
        mcp_servers=[mcp_server]
    )
    
    with trace("date_assistant"):
        result = await Runner.run(agent, request)
    
    display(Markdown(result.final_output))

Today's date is **August 14, 2025**, and in ISO format, it is **2025-08-14**.

### Step 3: Test Different Date Formats

In [6]:
# Test with a request for multiple formats
request = "Can you show me the current date in three different formats: US format (MM/DD/YYYY), European format (DD/MM/YYYY), and Unix timestamp?"

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(
        name="date_formatter", 
        instructions="You are a date formatting assistant. Use the available tools to show dates in the requested formats.", 
        model=model, 
        mcp_servers=[mcp_server]
    )
    
    with trace("date_formatter"):
        result = await Runner.run(agent, request)
    
    display(Markdown(result.final_output))

Here are the current date formats you requested:

- **US format (MM/DD/YYYY):** 08/14/2025
- **European format (DD/MM/YYYY):** 14/08/2025
- **Unix timestamp:** 1755171359

### Optional: Create an MCP Client for Native OpenAI Calls

This follows the pattern from `accounts_client.py` in your example.

In [ ]:
# date_client.py content (you would save this as a separate file)
date_client_code = '''
import asyncio
from typing import List, Dict, Any
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from openai import AsyncOpenAI
import os

async def list_date_tools():
    """List all available tools from the date server."""
    params = StdioServerParameters(
        command="uv",
        args=["run", "date_server.py"]
    )
    
    async with stdio_client(params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools = await session.list_tools()
            return tools.tools

async def get_date_tools_openai():
    """Convert MCP tools to OpenAI function tool format."""
    from openai.types.shared_params import FunctionTool
    
    mcp_tools = await list_date_tools()
    openai_tools = []
    
    for tool in mcp_tools:
        # Create a function that calls the MCP tool
        async def call_mcp_tool(tool_name=tool.name, **kwargs):
            params = StdioServerParameters(
                command="uv",
                args=["run", "date_server.py"]
            )
            
            async with stdio_client(params) as (read, write):
                async with ClientSession(read, write) as session:
                    await session.initialize()
                    result = await session.call_tool(tool_name, kwargs)
                    return result.content[0].text if result.content else "No result"
        
        # Convert to OpenAI format
        openai_tool = FunctionTool(
            name=tool.name,
            description=tool.description,
            params_json_schema={
                **tool.inputSchema,
                "additionalProperties": False
            },
            on_invoke_tool=lambda **kwargs, tn=tool.name: asyncio.create_task(call_mcp_tool(tn, **kwargs)),
            strict_json_schema=True
        )
        openai_tools.append(openai_tool)
    
    return openai_tools

async def use_with_openai():
    """Example of using the MCP tools directly with OpenAI."""
    client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    tools = await get_date_tools_openai()
    
    # Convert tools to OpenAI format for the API call
    tools_for_api = [
        {
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description,
                "parameters": tool.params_json_schema
            }
        }
        for tool in tools
    ]
    
    response = await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": "What is today\'s date?"}
        ],
        tools=tools_for_api,
        tool_choice="auto"
    )
    
    return response
'''

# Save this as date_client.py if you want to use it
with open('date_client.py', 'w') as f:
    f.write(date_client_code)

print("Created date_client.py for native OpenAI integration")

### Testing the Native OpenAI Client (Optional)

In [ ]:
# If you created the date_client.py file above, you can test it:
# from date_client import list_date_tools, get_date_tools_openai

# mcp_tools = await list_date_tools()
# print("MCP Tools:", mcp_tools)

# openai_tools = await get_date_tools_openai()
# print("\nOpenAI-formatted tools:", openai_tools)

## Summary

We've successfully created:

1. **`date_server.py`**: An MCP server that exposes tools to get the current date
   - `get_current_date`: Returns the current date in a readable format
   - `get_date_with_format`: Returns the date in a custom format

2. **This notebook**: Demonstrates how to:
   - Connect to the MCP server
   - Use it with an Agent
   - Test different date formatting requests

3. **`date_client.py`** (optional): A client wrapper for using the MCP tools with native OpenAI calls

The pattern follows the accounts example from your notebook, but with a simpler use case focused on date/time functionality.